# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных уже была сделана заранее.

In [1]:
#импорт библиотек
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

## Ознакомление с данными

Загружу предоставленный датасет и сохраню его в data.

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


У нас есть пять столбцов:

- сalls — количество звонков;
- minutes — суммарная длительность звонков в минутах;
- messages — количество sms-сообщений;
- mb_used — израсходованный интернет-трафик в Мб;
- is_ultra — тариф: «Ультра» — 1, «Смарт» — 0.

Все, кроме is_ultra являются вещественными, из них calls и messages выглядят как числа с плавающей точков, хотя фактически должны быть целочисленными. У нас не может быть 1,5 сообщения или звонка. Заменю в этих столбцах тип на int:

In [3]:
data[['calls', 'messages']] = data[['calls', 'messages']].astype('int64')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


Теперь оптимизирую хранение этих столбцов. Машинное обучение само по себе требует много вычислительной мощности для рассчётов и занимает много времени. Важно уменьшать объём занимаемых данных, если это возможно.

In [4]:
cols_to_float = ['minutes', 'mb_used']
for col in cols_to_float:
    data[col] = pd.to_numeric(data[col], downcast='float')

In [5]:
cols_to_int = ['calls', 'messages', 'is_ultra']
for col in cols_to_int:
    data[col] = pd.to_numeric(data[col], downcast='integer')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int16  
 1   minutes   3214 non-null   float32
 2   messages  3214 non-null   int16  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int8   
dtypes: float32(1), float64(1), int16(2), int8(1)
memory usage: 53.5 KB


Понизив выделяемое под хранение численных данных пространство, я уменьшила использование памяти в три раза.

На всякий случай, проверю наличие явных дубликатов и пропусков.

In [6]:
data.duplicated().sum()

0

Дубликатов в данных нет, проверим пропуски.

In [7]:
data.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

Пропусков также нет, датасет чист и готов к работе.

### Промежуточный вывод

В данном разделе я:
- Импортировала и рассмотрела предложенный датасет;
- Сменила формат с float на int в тех столбцах, где это имеет смысл;
- Уменьшила занимаемую память в три раза;
- Убедилась в отсутствии явных дубликатов и пропущенных значений.

## Выборки

В данном разделе я выделю из исходного датасета обучающую, валидационную и тестовую выборки.

Обучающая выборка используется для обучения, валидационная для проверки и корректировки в процессе подбора оптимальных гиперпараметров, а по тестовой я буду уже непосредственно проверять результат обучения.

In [8]:
data_train_valid, data_test = train_test_split(data, test_size=0.2, random_state=12345) #Отделяем сперва 1/5 под тест.выборку

In [9]:
data_train, data_valid = train_test_split(data_train_valid, test_size=0.25, random_state=12345)
print('data_train shape:', data_train.shape) #1/5 от исходного сета под валидационную выборку и 3/5 под обучающую
print('data_valid shape:', data_valid.shape)
print('data_test shape:', data_test.shape)

data_train shape: (1928, 5)
data_valid shape: (643, 5)
data_test shape: (643, 5)


Я разделила исходный датасет на три новых: обучающий, валидационный и тестовый, в соотношении 3 : 1 : 1.

Теперь выделю их features (параметры) и target (целевой параметр). Я буду учить модель определять целевой параметр по всем остальным.

In [10]:
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']
features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']
features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

### Промежуточный вывод

В данном разделе я выделила обучающую, валидационную и тестовую выборки, а также разделила их на признаки, по которым будет идти проверка, и цель, которую в ходе проверки машина будет определять.

## Выбор модели

В поставленной задаче сказано, что мне нужно построить модель для задачи классификации, которая будет предсказывать подходящий тариф: free или ultra, в зависимости от поведения пользователя.

Переберу три варианта модели:
- Решающее дерево (DecisionTreeClassifier);
- Случайный лес (RandomForestClassifier);
- Логическая регрессия (LogisticRegression)

Я обучу все три и затем выберу ту, которая покажет наивысший показатель точности (accuracy)

### Решающее дерево

Начнём с самого простого — решающего дерева. В процессе обучения, дерево будет наращивать "ветви", в которых в зависимости от параметров будет выдавать 1 или 0, пока в конце концов не дойдёт до конечного решения.

Метод DecisionTreeClassifier у меня принимает два параметра: random_state нужен для повторяемости результатов обучения, чтобы при каждом новом запуске кода я получала одни и те же ответы от модели, а mac_depth определяет максимальную глубину, которую может иметь дерево решений.

In [11]:
model_tree = None
best_accuracy = 0
best_depth = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions_valid)
    if accuracy > best_accuracy:
        model_tree = model
        best_accuracy = accuracy
        best_depth = depth

print('Решающее дерево с глубиной ', best_depth, 'показало лучшую точность ', best_accuracy)

Решающее дерево с глубиной  7 показало лучшую точность  0.7744945567651633


В ячейке выше я перебирала глубину дерева (количество итераций, за которое дерево даёт ответ) от 1 до 10, чтобы определить оптимальную глубину. Лучшее значение accuracy показало дерево с глубиной в 7 уровней, его точность составляет 0.77. Минимальная приемлемая точность у нас 0.75, так что это дерево подходит. И всё же, я наверняка могу получить лучший результат с другими моделями, так что рассмотрим и их тоже.

### Случайный лес

Случайный лес представляет собой набор из решающих деревьев, каждое из которых проводит рассчёт и даёт свой ответ, после чего все они методом голосования сводятся к среднему, на основе которого и решается конечный ответ всего леса. У случайного леса принцип действия, а следовательно и гиперпараметры схожи с таковыми у решающего дерева. Дополнительно я укажу только n_estimators, определяющий количество деревьев в лесу.

In [12]:
model_forest = None
best_accuracy = 0
best_est = 0
best_depth = 0
for est in range(10, 101, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predictions_valid)
        if accuracy > best_accuracy:
            model_forest = model
            best_accuracy = accuracy
            best_est = est
            best_depth = depth

print ('Случайный лес с количеством деревьев', best_est, 'и их глубиной', best_depth, 'показал лучшую точность', best_accuracy)

Случайный лес с количеством деревьев 50 и их глубиной 10 показал лучшую точность 0.7978227060653188


В ячейке выше я перебираю несколько "лесов", с количеством деревьев от 10 до 100 с шагом в 10. Из них, самую высокую точность показал лес с количеством деревьев 50, глубиной 10. Точность получилась почти 0.8, что значительно выше, чем у одинокого решающего дерева.

### Логическая регрессия

Теперь рассмотрю модель логической регрессии. Она рассматривает каждый объект в зависимости от его расстояния до центра классов (в нашем случае классов два: ultra и free) и, смотря к какому из классов объект ближе, выдаёт ответ. Гиперпараметр max_iter определяет лимит итераций, которые мы разрешаем сделать модели в ходе вычисления ответа.

In [13]:
model_reg = LogisticRegression(random_state=12345, max_iter=1000)
model_reg.fit(features_train, target_train)
predictions_valid = model_reg.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions_valid)
print('Логическая регрессия показала точность ', accuracy)

Логическая регрессия показала точность  0.7262830482115086


Логическая регрессия — очень быстрая модель, не в сравнение с медленным и громоздким случайным лесом. Но в данном случае она показала самый низкий результат — всего 0.73 по сравнению с 0.77 и 0.8 для двух предыдущих. Это ниже, чем минимальная требуемая точность по заданию.

### Промежуточный вывод

Самую низкую точность показала логическая регрессия, всего 0.73, что даже ниже требуемой 0.75. Оптимальный результат дало решающее дерево, её точность равно 0.77. Самое высокое значение точности у случайного леса с количеством деревьев 50 и их максимальной глубиной 10, его точность равна почти 0.8.

## Проверка модели

Теперь, когда я выбрала подходящую модель, проверю её на тестовой выборке.

В предыдущем разделе я обучила модель методом "случайный лес" и определили оптимальные гиперпараметры для неё. Воспользуюсь ею для предсказания значений столбца is_ultra для тесторовой выборки. После чего сравню результат с реальными значениями is_ultra и выведу на экран показатель accuracy.

In [14]:
model = RandomForestClassifier(random_state=12345, n_estimators=best_est, max_depth=best_depth)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
accuracy = accuracy_score(target_test, predictions_test)
print ('Точность предсказаний по тестовой выборке равна', accuracy)

Точность предсказаний по тестовой выборке равна 0.7993779160186625


### Промежуточный вывод

Модель показала точность по тестовой выборке даже чуть выше чем по валидационной: 0.799 против 0.798. Это вполне высокий результат, с учётом, что минимальное требование в точности у нас от 0.75.

## Вывод

В данном проекте по машинному обучению задачей было построить модель для классификации пользователей по тарифам: ultra и free. Модель должна по поведению пользователя с точностью не ниже 75% определять, к какому из двух тарифов он относится.

Для выполнения данной задачи я произвела следующие действия:
- Оптимизировала хранимые данные в исходном датасете;
- Выделила три выборки: обучающую, валидационную и тестовую;
- Определила модель с наибольшим значением accuracy и обучила её;
- Проверила обученную модель на тестовых данных.

Используя "случайный лес" я обучила модель, которая в 80% случаев верно определяет, каким тарифом пользуется человек, исходя из его поведения. У "леса" низкая скорость работы из-за большого количества моделей внутри него, но поскольку для бизнеса главное — максимально высокая точность, это оказалось наиболее приемлемым вариантом.